# Imports

In [4]:
# !pip install a2pm

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
import torch

import time

In [2]:
from a2pm import A2PMethod
from a2pm.callbacks import BaseCallback, MetricCallback, TimeCallback
from a2pm.patterns import BasePattern, CombinationPattern, IntervalPattern
from a2pm.wrappers import BaseWrapper, KerasWrapper, SklearnWrapper, TorchWrapper

In [62]:
# X_train = np.load('X-IIoT-pre-processed/x_train.npy')
# y_train = np.load('X-IIoT-pre-processed/y_train.npy')
# X_val = np.load('X-IIoT-pre-processed/x_val.npy')
# y_val = np.load('X-IIoT-pre-processed/y_val.npy')
# X_test = np.load('X-IIoT-pre-processed/x_test.npy')
# y_test = np.load('X-IIoT-pre-processed/y_test.npy')

X_test = np.load('/home/jovyan/Wustl_iiot/x_test.npy')
X_train = np.load('/home/jovyan/Wustl_iiot/x_train.npy')
X_val = np.load('/home/jovyan/Wustl_iiot/x_val.npy')
y_test = np.load('/home/jovyan/Wustl_iiot/y_test.npy')
y_train = np.load('/home/jovyan/Wustl_iiot/y_train.npy')
y_val = np.load('/home/jovyan/Wustl_iiot/y_val.npy')

# Train

We train the baseline model to get an impression of how our model perform on an IDS dataset without any adversarial samples.

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [5]:
model_rf = RandomForestClassifier()

param_grid = {
    # 'n_estimators': [10, 50, 100],
    # 'max_depth': [None, 50, 100],
    'n_estimators': [10],
    'max_depth': [None],
}

grid_search_rf = GridSearchCV(model_rf, param_grid, cv=3, scoring='accuracy', verbose=3)
grid_search_rf.fit(X_train, y_train)

print("Best parameters found: ", grid_search_rf.best_params_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3] END ...max_depth=None, n_estimators=10;, score=1.000 total time=   4.1s
[CV 2/3] END ...max_depth=None, n_estimators=10;, score=1.000 total time=   3.8s
[CV 3/3] END ...max_depth=None, n_estimators=10;, score=1.000 total time=   6.0s
[CV 1/3] END ...max_depth=None, n_estimators=50;, score=1.000 total time=  21.7s
[CV 2/3] END ...max_depth=None, n_estimators=50;, score=1.000 total time=  22.6s
[CV 3/3] END ...max_depth=None, n_estimators=50;, score=1.000 total time=  23.0s
[CV 1/3] END ..max_depth=None, n_estimators=100;, score=1.000 total time=  50.8s
[CV 2/3] END ..max_depth=None, n_estimators=100;, score=1.000 total time=  48.4s
[CV 3/3] END ..max_depth=None, n_estimators=100;, score=1.000 total time=  49.5s
[CV 1/3] END .....max_depth=50, n_estimators=10;, score=1.000 total time=   5.7s
[CV 2/3] END .....max_depth=50, n_estimators=10;, score=1.000 total time=   5.4s
[CV 3/3] END .....max_depth=50, n_estimators=10;,

# Evaluate

Evaluate the RF model on the IDS dataset, once again with no adversarial samples.

## Helpers

In [5]:


def metrics_master(model, X_test, y_test, type):
    metrics_weighted(model, X_test, y_test, type)
    metrics_macro(model, X_test, y_test, type)

def metrics_macro(model, X_test, y_test, type):
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    data = {
    "Metric": ["Accuracy", "Macro Precision", "Macro Recall", "Macro F1 Score"],
    type: [accuracy, precision, recall, f1]
    }
    
    # Creating DataFrame
    df = pd.DataFrame(data)
    
    # Saving to CSV
    csv_file_path = "/home/jovyan/A2PM/metrics.csv"
    df.to_csv(csv_file_path, mode='a', index=False)

    print(type)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Macro Precision: {precision * 100:.2f}%")
    print(f"Macro Recall: {recall * 100:.2f}%")
    print(f"Macro F1 Score: {f1 * 100:.2f}%")
    
def metrics_weighted(model, X_test, y_test, type):
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    data = {
    "Metric": ["Accuracy", "Weighted Precision", "Weighted Recall", "Weighted F1 Score"],
    type: [accuracy, precision, recall, f1]
    }
    
    # Creating DataFrame
    df = pd.DataFrame(data)
    
    # Saving to CSV
    csv_file_path = "/home/jovyan/A2PM/metrics.csv"
    df.to_csv(csv_file_path, mode='a', index=False)

    print(type)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Weighted Precision: {precision * 100:.2f}%")
    print(f"Weighted Recall: {recall * 100:.2f}%")
    print(f"Weighted F1 Score: {f1 * 100:.2f}%")

## Scores

In [53]:
rf_model = grid_search_rf.best_estimator_
metrics_master(rf_model, X_test, y_test)

Accuracy: 99.95%
Weighted Precision: 99.97%
Weighted Recall: 99.95%
Weighted F1 Score: 99.96%
Accuracy: 99.95%
Macro Precision: 86.10%
Macro Recall: 95.38%
Macro F1 Score: 87.74%


# A2PM Generation

Generate A2PM adversarial samples for further experiments

Since we know the data is pre-processed, the categorical variables will be binary columns, and conversely the numerical variables will be the others. Let's find these columns.

In [65]:
def find_binary_columns(X_train):
    binary_columns = []
    for col in range(X_train.shape[1]):
        unique_values = np.unique(X_train[:, col])
        if set(unique_values).issubset({0, 1}):
            binary_columns.append(col)
    return binary_columns

binary_columns = find_binary_columns(X_train)

numerical_columns = []
for i in range(0,42):
    if i not in binary_columns:
        numerical_columns.append(i)

In [66]:
X_train.shape

(716256, 42)

The following configuration is the boilerplate found on the A2PM github (https://github.com/vitorinojoao/a2pm). The only modifications are that the interval pattern is applied to the numerical columns, and the combination pattern is applied to the categorical (binary) variables. I removed integer features since the numerical columns are float values, and removed locked_features as I wasn't sure which features needed to be kept static (data was given as processed .npy files)

In [67]:
RF_base = grid_search_rf.best_estimator_

classifier = SklearnWrapper(RF_base)


# rule of thumb: Interval for numerical, combination for categorical
pattern = (
    {
        "type": "interval",
        "features": numerical_columns,
        #"integer_features": list(range(10, 20)),
        "ratio": 0.1,
        "max_ratio": 0.3,
        "missing_value": 0.0,
        "probability": 1,
    },
    # {
    #     "type": "combination",
    #     "features": binary_columns,
    #     #"locked_features": list(range(30, 40)), # Locks some features to ensure validity. Not using this because data is .npy and unreadable
    #     "probability": 0.4,
    # },
)

method = A2PMethod(pattern)

We will generate attacks based off the entire dataset. We will resplit the train-test sets afterward in our experiments

In [68]:
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

In [26]:
start_time = time.time()

X_adversarial = method.fit_generate(classifier, X, y)

training_time = time.time() - start_time
print(f"Attack Time: {training_time}")

Attack Time: 401.11300897598267


# Scores after A2PM Attack

Here we are evaluating how our trained (unprotected) RF model performs on the full A2PM dataset. Each sample is now perturbed, but should still correspond to the original target classes y

In [29]:
metrics_master(rf_model, X_adversarial, y)

Accuracy: 9.98%
Weighted Precision: 87.16%
Weighted Recall: 9.98%
Weighted F1 Score: 13.38%
Accuracy: 9.98%
Macro Precision: 34.59%
Macro Recall: 48.23%
Macro F1 Score: 19.43%


# Adversarial Training Preprocess

As per many adversarial defense papers, adversarial training has been cited as one of the best ways to improve resilience. We will retrain our model with the original data, in addition to an added 10% of the data being adversarial samples with an "adversarial" class (class 19).

In [33]:
X_train.shape[0]+X_test.shape[0]

955009

In [35]:

adv_samples_num = int(955009*0.10/0.9) 
adv_samples_num # 10%

106112

In [36]:
X_adversarial_indices = np.random.choice(X_adversarial.shape[0], size=adv_samples_num, replace=False)
X_adversarial_sampled = X_adversarial[X_adversarial_indices]

y_adv = np.full(adv_samples_num, 5)
y_adv

array([5, 5, 5, ..., 5, 5, 5])

In [37]:
np.unique(y_train)

array([0, 1, 2, 3, 4])

In [38]:
X_new = X.copy()
y_new = y.copy()

X_combined = np.vstack((X_new, X_adversarial_sampled))
y_combined = np.concatenate((y_new, y_adv))

# Adversarial Training

Our whole data now contains 10% A2PM samples. Assuming a uniformly random 80/20 split, our training data will be expected to contain 8% A2PM samples, while the testing data is expected to contain 2% A2PM samples. 

In [39]:
X_train_adv, X_test_adv, y_train_adv, y_test_adv = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

In [40]:
model_rf_protected = RandomForestClassifier()

param_grid = {
    # 'n_estimators': [10, 50, 100],
    # 'max_depth': [None, 50, 100],
    'n_estimators': [10],
    'max_depth': [None],
}

grid_search_rf_protected = GridSearchCV(model_rf_protected, param_grid, cv=3, scoring='accuracy', verbose=3)
grid_search_rf_protected.fit(X_train_adv, y_train_adv)

print("Best parameters found: ", grid_search_rf.best_params_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3] END ...max_depth=None, n_estimators=10;, score=1.000 total time=  12.2s
[CV 2/3] END ...max_depth=None, n_estimators=10;, score=1.000 total time=  11.7s
[CV 3/3] END ...max_depth=None, n_estimators=10;, score=1.000 total time=  11.3s
[CV 1/3] END ...max_depth=None, n_estimators=50;, score=1.000 total time=  56.2s
[CV 2/3] END ...max_depth=None, n_estimators=50;, score=1.000 total time=  57.2s
[CV 3/3] END ...max_depth=None, n_estimators=50;, score=1.000 total time=  58.9s
[CV 1/3] END ..max_depth=None, n_estimators=100;, score=1.000 total time= 1.9min
[CV 2/3] END ..max_depth=None, n_estimators=100;, score=1.000 total time= 1.8min
[CV 3/3] END ..max_depth=None, n_estimators=100;, score=1.000 total time= 1.9min
[CV 1/3] END .....max_depth=50, n_estimators=10;, score=1.000 total time=  12.3s
[CV 2/3] END .....max_depth=50, n_estimators=10;, score=1.000 total time=  11.5s
[CV 3/3] END .....max_depth=50, n_estimators=10;,

In [41]:
rf_model_protected = grid_search_rf_protected.best_estimator_

# Scores (Binary A2PM Detection after Adversarial Training)

We want to see how well our protected model can detect A2PM samples. We run it against the unseen A2PM set

In [42]:
X_adversarial_unseen = X_adversarial[np.logical_not(np.isin(np.arange(X_adversarial.shape[0]), X_adversarial_indices))]
len(X_adversarial_unseen)

848897

In [43]:
protected_adversarial_preds = rf_model_protected.predict(X_adversarial_unseen)

All we need to know is if the model correctly predicts every sample as class 19 (A2PM), and as such we will treat this as a binary classification problem.

In [48]:
y_adv_full = np.full(len(X_adversarial_unseen), 5)

positive_class = 5
y_test_binary = (y_adv_full == positive_class).astype(int)
adversarial_preds_binary = (protected_adversarial_preds == positive_class).astype(int)

In [49]:
adversarial_accuracy = accuracy_score(y_test_binary, adversarial_preds_binary)
adversarial_precision = precision_score(y_test_binary, adversarial_preds_binary, average='binary')
adversarial_recall = recall_score(y_test_binary, adversarial_preds_binary, average='binary')
adversarial_f1 = f1_score(y_test_binary, adversarial_preds_binary, average='binary')

print(f"Adversarial Accuracy: {adversarial_accuracy * 100:.2f}%")
print(f"Adversarial Precision: {adversarial_precision * 100:.2f}%")
print(f"Adversarial Recall: {adversarial_recall * 100:.2f}%")
print(f"Adversarial F1 Score: {adversarial_f1 * 100:.2f}%")

Adversarial Accuracy: 100.00%
Adversarial Precision: 100.00%
Adversarial Recall: 100.00%
Adversarial F1 Score: 100.00%


# Scores (10% A2PM Samples, no Adv Training)

Here we benchmark how well our unprotected model performs on a test set with 10% A2PM samples and 90% real samples

In [54]:
metrics_master(rf_model, X_test_adv, y_test_adv)

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 89.94%
Weighted Precision: 88.67%
Weighted Recall: 89.94%
Weighted F1 Score: 89.27%
Accuracy: 89.94%
Macro Precision: 45.74%
Macro Recall: 80.77%
Macro F1 Score: 49.31%


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Scores (10% A2PM Samples w/ Adv Training)

Here we benchmark how well our protected model performs on a test set with 10% A2PM samples and 90% real samples

In [55]:
metrics_master(rf_model_protected, X_test_adv, y_test_adv)

Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
Accuracy: 100.00%
Macro Precision: 97.54%
Macro Recall: 97.96%
Macro F1 Score: 97.71%


In [59]:
def find_binary_columns(X_train):
    binary_columns = []
    for col in range(X_train.shape[1]):
        unique_values = np.unique(X_train[:, col])
        if set(unique_values).issubset({0, 1}):
            binary_columns.append(col)
    return binary_columns

binary_columns = find_binary_columns(X_train)

numerical_columns = []
for i in range(0,42):
    if i not in binary_columns:
        numerical_columns.append(i)

RF_base = grid_search_rf.best_estimator_

classifier = SklearnWrapper(RF_base)


ratio_values = [0.01, 0.05, 0.1, 0.2, 0.3]
# rule of thumb: Interval for numerical, combination for categorical
for ratio in ratio_values:
    pattern = (
        {
            "type": "interval",
            "features": numerical_columns,
            #"integer_features": list(range(10, 20)),
            "ratio": ratio,
            "max_ratio": 0.3,
            "missing_value": 0.0,
            "probability": 1,
        },
        # {
        #     "type": "combination",
        #     "features": binary_columns,
        #     #"locked_features": list(range(30, 40)), # Locks some features to ensure validity. Not using this because data is .npy and unreadable
        #     "probability": 0.4,
        # },
    )
    
    method = A2PMethod(pattern)
    
    X = np.concatenate((X_train, X_test), axis=0)
    y = np.concatenate((y_train, y_test), axis=0)
    
    start_time = time.time()
    
    X_adversarial = method.fit_generate(classifier, X, y)
    
    training_time = time.time() - start_time
    print(f"Attack Time: {training_time}")

    text = f"{ratio}:full A2PM Attack"
    metrics_master(rf_model, X_adversarial, y, text)
    
    
    adv_samples_num = int(955009*0.10/0.9) 
    
    X_adversarial_indices = np.random.choice(X_adversarial.shape[0], size=adv_samples_num, replace=False)
    X_adversarial_sampled = X_adversarial[X_adversarial_indices]
    
    y_adv = np.full(adv_samples_num, 5)
    
    X_new = X.copy()
    y_new = y.copy()
    
    X_combined = np.vstack((X_new, X_adversarial_sampled))
    y_combined = np.concatenate((y_new, y_adv))
    
    X_train_adv, X_test_adv, y_train_adv, y_test_adv = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)
    
    model_rf_protected = RandomForestClassifier()
    
    param_grid = {
        # 'n_estimators': [10, 50, 100],
        # 'max_depth': [None, 50, 100],
        'n_estimators': [10],
        'max_depth': [None],
    }
    
    grid_search_rf_protected = GridSearchCV(model_rf_protected, param_grid, cv=3, scoring='accuracy', verbose=3)
    grid_search_rf_protected.fit(X_train_adv, y_train_adv)
    
    print("Best parameters found: ", grid_search_rf.best_params_)
    
    rf_model_protected = grid_search_rf_protected.best_estimator_

    text = f"{ratio}:no Adv Training"
    metrics_master(rf_model, X_test_adv, y_test_adv, text)
    text = f"{ratio}:w/ Adv Training"
    metrics_master(rf_model_protected, X_test_adv, y_test_adv, text)

Attack Time: 405.23783826828003
0.01:full A2PM Attack
Accuracy: 10.95%
Weighted Precision: 89.84%
Weighted Recall: 10.95%
Weighted F1 Score: 15.05%
0.01:full A2PM Attack
Accuracy: 10.95%
Macro Precision: 35.14%
Macro Recall: 50.14%
Macro F1 Score: 19.89%
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END ...max_depth=None, n_estimators=10;, score=1.000 total time=  11.2s
[CV 2/3] END ...max_depth=None, n_estimators=10;, score=1.000 total time=  11.2s
[CV 3/3] END ...max_depth=None, n_estimators=10;, score=1.000 total time=  12.2s
Best parameters found:  {'max_depth': None, 'n_estimators': 10}


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.01:no Adv Training
Accuracy: 89.94%
Weighted Precision: 88.60%
Weighted Recall: 89.94%
Weighted F1 Score: 89.23%


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.01:no Adv Training
Accuracy: 89.94%
Macro Precision: 45.33%
Macro Recall: 80.77%
Macro F1 Score: 48.72%
0.01:w/ Adv Training
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.01:w/ Adv Training
Accuracy: 100.00%
Macro Precision: 98.08%
Macro Recall: 98.68%
Macro F1 Score: 98.29%
Attack Time: 397.6914293766022
0.05:full A2PM Attack
Accuracy: 10.44%
Weighted Precision: 88.59%
Weighted Recall: 10.44%
Weighted F1 Score: 14.18%
0.05:full A2PM Attack
Accuracy: 10.44%
Macro Precision: 34.95%
Macro Recall: 48.80%
Macro F1 Score: 19.72%
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END ...max_depth=None, n_estimators=10;, score=1.000 total time=  12.1s
[CV 2/3] END ...max_depth=None, n_estimators=10;, score=1.000 total time=  10.9s
[CV 3/3] END ...max_depth=None, n_estimators=10;, score=1.000 total time=  11.2s
Best parameters found:  {'max_depth': None, 'n_estimators': 10}


...

0.2:no Adv Training
Accuracy: 89.94%
Macro Precision: 46.18%
Macro Recall: 80.77%
Macro F1 Score: 49.22%
0.2:w/ Adv Training
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.2:w/ Adv Training
Accuracy: 100.00%
Macro Precision: 98.06%
Macro Recall: 98.24%
Macro F1 Score: 98.05%
Attack Time: 336.6277868747711
0.3:full A2PM Attack
Accuracy: 9.49%
Weighted Precision: 79.94%
Weighted Recall: 9.49%
Weighted F1 Score: 12.96%
0.3:full A2PM Attack
Accuracy: 9.49%
Macro Precision: 31.51%
Macro Recall: 42.22%
Macro F1 Score: 17.70%
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END ...max_depth=None, n_estimators=10;, score=1.000 total time=  12.7s
[CV 2/3] END ...max_depth=None, n_estimators=10;, score=1.000 total time=  12.3s
[CV 3/3] END ...max_depth=None, n_estimators=10;, score=1.000 total time=  12.3s
Best parameters found:  {'max_depth': None, 'n_estimators': 10}


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.3:no Adv Training
Accuracy: 89.94%
Weighted Precision: 88.66%
Weighted Recall: 89.94%
Weighted F1 Score: 89.27%


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.3:no Adv Training
Accuracy: 89.94%
Macro Precision: 46.39%
Macro Recall: 80.77%
Macro F1 Score: 49.48%
0.3:w/ Adv Training
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.3:w/ Adv Training
Accuracy: 100.00%
Macro Precision: 99.29%
Macro Recall: 99.12%
Macro F1 Score: 99.19%


In [ ]:
ratio_values = [0.01,0.05,0.1,0.2,0.3]
for ratio in ratio_values:
    pattern = (
    {
        "type": "interval",
        "features": numerical_columns,
        #"integer_features": list(range(10, 20)),
        "ratio": ratio,
        "max_ratio": 0.3,
        "missing_value": 0.0,
        "probability": 1,
    },
    # {
    #     "type": "combination",
    #     "features": binary_columns,
    #     #"locked_features": list(range(30, 40)), # Locks some features to ensure validity. Not using this because data is .npy and unreadable
    #     "probability": 0.4,
    # },
    )

    method = A2PMethod(pattern)

    start_time = time.time()
    
    X_adversarial = method.fit_generate(X, y)
    
    training_time = time.time() - start_time
    print(f"Attack Time: {training_time}")

    path = f"A2PM_{ratio}"
    np.save('/home/jovyan/A2PM/array.npy', X_adversarial)

In [6]:
X_test = np.load('/home/jovyan/Wustl_iiot/x_test.npy')
X_train = np.load('/home/jovyan/Wustl_iiot/x_train.npy')
X_val = np.load('/home/jovyan/Wustl_iiot/x_val.npy')
y_test = np.load('/home/jovyan/Wustl_iiot/y_test.npy')
y_train = np.load('/home/jovyan/Wustl_iiot/y_train.npy')
y_val = np.load('/home/jovyan/Wustl_iiot/y_val.npy')


ratio_values = [0.01, 0.05, 0.1, 0.2, 0.3]
# rule of thumb: Interval for numerical, combination for categorical
for ratio in ratio_values:
    xtext = f"/home/jovyan/A2PM/A2PM_adv_sample/A2PM_{ratio}_fullattack_X.npy"
    X_adv = np.load(xtext)
    ytext = f"/home/jovyan/A2PM/A2PM_adv_sample/A2PM_{ratio}_fullattack_y.npy"
    y_adv = np.load(ytext)
    
    X_train_adv, X_test_adv, y_train_adv, y_test_adv = train_test_split(X_adv, y_adv, test_size=0.2, random_state=42)
    
    
    rf = RandomForestClassifier()
    rf = rf.fit(X_train, y_train)
    
    text=f"{ratio}_origin"
    metrics_master(rf, X_test, y_test, text)
    
    text=f"{ratio}_test attack dataset"
    metrics_master(rf, X_test_adv, y_test_adv, text)
    
    rf_def = RandomForestClassifier()
    rf_def = rf_def.fit(X_train_adv, y_train_adv)
    text=f"{ratio}_defensed model"
    metrics_master(rf_def, X_test_adv, y_test_adv, text)

0.01_origin
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.01_origin
Accuracy: 100.00%
Macro Precision: 99.09%
Macro Recall: 99.23%
Macro F1 Score: 99.14%
0.01_test attack dataset
Accuracy: 99.99%
Weighted Precision: 99.99%
Weighted Recall: 99.99%
Weighted F1 Score: 99.99%
0.01_test attack dataset
Accuracy: 99.99%
Macro Precision: 99.97%
Macro Recall: 98.27%
Macro F1 Score: 99.08%
0.01_defensed model
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.01_defensed model
Accuracy: 100.00%
Macro Precision: 100.00%
Macro Recall: 100.00%
Macro F1 Score: 100.00%
0.05_origin
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.05_origin
Accuracy: 100.00%
Macro Precision: 99.09%
Macro Recall: 99.23%
Macro F1 Score: 99.14%
0.05_test attack dataset
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.05_